In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.pyplot import plot, draw, show
from scipy.linalg import lu
import scipy.linalg as la
import random
import simpy
import pandas as pd
import math
from IPython.display import display, HTML
import time
# import progressbar

In [ ]:
# Init seed
RANDOM_SEED = 246
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Init service schemes
serviceScheme = "Long-Tailed"  # Can be "Long-Tailed", "Poisson", "Deterministic", "Inverse", "Random"
serviceSchemes = [
#     "Exponential",
    "Poisson",
#     "Deterministic",  
#     "Inverse", 
#     "Random", 
#     "Long-Tailed"
]

# Init simulation specific parameters
N_helpers = [1, 2, 4]  # Number of machines in the queue
# N_helpers = [1, 2]
SIM_TIME = 80  # Simulation time in minutes
NRUNS = 1  # Amount of runs
chanceLongTail = 0.75  # Chance of longtail lowerbound
lI = 8.0 / 5
lambdaIAT = [N * lI for N in N_helpers]  # Create a customer every ~lI minutes
print(lambdaIAT)
cC = 20000
customerCount = [N * cC for N in N_helpers]  # Amount of customers
print(customerCount)
rL = 1000
runLength = [N * rL for N in N_helpers]  # Amount of customer per average block
print(runLength)

# Init servicetimes
serviceTime = [2 for N in N_helpers
               ]  # Minutes it takes to help a customer / Deterministic
ltLow = [1.65 for N in N_helpers]  # Lower bound long-tailed distribution
ltHigh = [5 for N in N_helpers]  # Upper bound long-tailed distribution
randspread = [1 for N in N_helpers]  # Spread of the random distribution
invMax = [45
          for N in N_helpers]  # max waiting time for the inverse servicetime
qServeInv = [[
    i for i in range(invMax[queueIndex], -1, -1)
    for x in range(math.ceil(customerCount[queueIndex] / invMax[queueIndex]))
] for queueIndex in range(len(N_helpers))]

# print(f"Rho Deterministic:\t {(lambdaIAT[0]) / (N * (1/serviceTime[0]))}")
# print(f"Rho Poisson/random:\t {(lambdaIAT[0]) / (N * (1/serviceTime[0]))}")
# print(f"Rho Inverse:\t {(lambdaIAT[0]) / (N * (1/np.mean(serviceTime)))}")
# print(f"Rho Long-Tailed:\t {(lambdaIAT[0]) / (N * (1/(0.75*ltL + 0.25*ltH)))}")

# Choose your queueing system
# resources = [simpy.PriorityResource]
# resources = [simpy.Resource]
resources = [simpy.PriorityResource, simpy.Resource]


class Queue(object):
    def __init__(self, env, N, resource, helperIndex):
        self.env = env
        self.machine = resource(env, N)
        self.customerHelped = 0
        self.serverN = N
        self.helperIndex = helperIndex

    def helped(self, customer, customerServe):
        yield self.env.timeout(customerServe)


def checkTest(now, entertime, customerHelped, runLength, customerLeft, cw):
    #     print(runLength)
    #     print(customerHelped % runLength)
    #     print(subWaitingList)
    #     print(f"Now {now},   entertime {entertime},       difference {now-entertime}")1
    if customerLeft % runLength == 0:
        if customerLeft != 0:
            if len(cw.subWaitingList) != 0:
#                 print(cw.subWaitingList)
                submean = np.mean(cw.subWaitingList)
                del cw.subWaitingList[:]
                return submean
        del cw.subWaitingList[:]
    elif 0 < customerLeft % runLength <= 0.3 * runLength:
        pass
    elif 0.3 * runLength < customerLeft % runLength < runLength:
        cw.subWaitingList.append(now - entertime)
    return None


def customer(env, cw, resource, serviceScheme, runLength, arrivaltime):
#     global customerLeft
    if serviceScheme == "Long-Tailed":
        rCheck = random.random()
        if rCheck <= 0.75:
            customerServeTime = np.random.poisson(1.0 / ltLow[cw.helperIndex])
        else:
            customerServeTime = np.random.poisson(1.0 / ltHigh[cw.helperIndex])
    elif serviceScheme == "Poisson":
        customerServeTime = np.random.poisson(
            1.0 / serviceTime[cw.helperIndex])
    elif serviceScheme == "Random":
        customerServeTime = random.uniform(
            serviceTime[cw.helperIndex] - randspread[cw.helperIndex],
            serviceTime[cw.helperIndex] + randspread[cw.helperIndex])
    elif serviceScheme == "Deterministic":
        customerServeTime = 1 / serviceTime[cw.helperIndex]
    elif serviceScheme == "Inverse":
        customerServeTime = qServeInv[cw.helperIndex][cw.customerHelped]
    elif serviceScheme == "Exponential":
        customerServeTime = random.expovariate(serviceTime[cw.helperIndex])


#         print(customerServeTime)
    

    customerServeTime = 0.000001 if customerServeTime == 0 else customerServeTime

#     tmpRho.append(
#         lambdaIAT[cw.helperIndex] / (cw.serverN * (1 / customerServeTime)))
    tmpRho.append(
        arrivaltime / (cw.serverN * (1 / customerServeTime)))
#     print(arrivaltime)

    
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerServeTime) as request:
            request.time = customerServeTime
            enterQueue = env.now
            yield request
            outQueue = env.now
            cw.customerLeft += 1
#             print(f"customerLeft = {cw.customerLeft}")

            #             tmpWait.append(env.now - enterQueue)
            blockMean = checkTest(outQueue, enterQueue, cw.customerHelped,
                                  runLength, cw.customerLeft, cw)
            if blockMean != None:
                tmpWait.append(blockMean)
            tmpServe.append(customerServeTime)
            yield env.process(cw.helped(cw.customerHelped, customerServeTime))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.time = customerServeTime
            enterQueue = env.now
            yield request
            outQueue = env.now
            cw.customerLeft += 1

            #             tmpWait.append(env.now - enterQueue)
            if cw.customerHelped > runLength:
                blockMean = checkTest(outQueue, enterQueue, cw.customerHelped,
                                  runLength, cw.customerLeft, cw)
            else:
                blockMean = None
            if blockMean != None:
#                 print(blockMean)
                tmpWait.append(blockMean)
            tmpServe.append(customerServeTime)
            yield env.process(cw.helped(cw.customerHelped, customerServeTime))


def setup(env, N, lambdaIAT, customerCount, resource, serviceScheme,
          helperIndex, runLength, customerLeft, subWaitingList):
    queue = Queue(env, N, resource, helperIndex)
    queue.customerLeft = customerLeft
    queue.subWaitingList = subWaitingList
    # Create more customers while the simulation is running
    s = np.random.poisson(1 / lambdaIAT, customerCount)
    while queue.customerHelped < customerCount:
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(customer(env, queue, resource, serviceScheme, runLength, 
                             s[queue.customerHelped - 1]))


columns = [
    "Rho", "Average", "Variance", "std_dev", "Resource", "Helpers", "run",
    "serviceScheme", "ServeMean"
]

# bariterations = len(serviceSchemes) * len(resources) * len(N_helpers)
# bar = progressbar.ProgressBar(max_value=bariterations)

resourceStatsRun = pd.DataFrame(columns=columns)
customerAll = pd.DataFrame(columns=columns)
for serviceScheme in serviceSchemes:
    for resource in resources:
        for helperIndex, N in enumerate(N_helpers):
            runServe = []
            runWait = []
            runRho = []
            for j in range(NRUNS):
                random.seed(RANDOM_SEED)
                np.random.seed(RANDOM_SEED)
                subWaitingList = []
                customerLeft = 0
                print(
                    f"Run {j} for {N} helpers for resource {resource} and service {serviceScheme}."
                )
                tmpWait = []
                tmpServe = []
                tmpRho = []
                env = simpy.Environment()
                env.process(
                    setup(env, N, lambdaIAT[helperIndex],
                          customerCount[helperIndex], resource, serviceScheme,
                          helperIndex, runLength[helperIndex], customerLeft, 
                          subWaitingList))
                env.run()

                resourceStatsRun.loc[serviceScheme + "_" + str(resource)[33:-2]
                                     + "_" + str(j) + "_" + str(N)] = [
                                         np.mean(tmpRho),
                                         np.mean(tmpWait),
                                         np.var(tmpWait),
                                         np.std(tmpWait),
                                         str(resource)[33:-2], N, j,
                                         serviceScheme,
                                         np.mean(tmpServe)
                                     ]

                runWait.append(tmpWait)
                runServe.append(tmpServe)
                runRho.append(tmpRho)

            np.array(runWait).flatten()
            np.array(runServe).flatten()
            np.array(runRho).flatten()
            #             print(runRho)
            customerAll.loc[serviceScheme + "_" + str(resource)[33:-2] + "_" +
                            str(N)] = [
                                np.mean(runRho),
                                np.mean(runWait),
                                np.var(runWait),
                                np.std(runWait),
                                str(resource)[33:-2], N, None, serviceScheme,
                                np.mean(runServe)
                            ]
#             bar.update(1)

[1.6, 3.2, 6.4]
[20000, 40000, 80000]
[1000, 2000, 4000]
Run 0 for 1 helpers for resource <class 'simpy.resources.resource.PriorityResource'> and service Poisson.
Run 0 for 2 helpers for resource <class 'simpy.resources.resource.PriorityResource'> and service Poisson.
Run 0 for 4 helpers for resource <class 'simpy.resources.resource.PriorityResource'> and service Poisson.
Run 0 for 1 helpers for resource <class 'simpy.resources.resource.Resource'> and service Poisson.
Run 0 for 2 helpers for resource <class 'simpy.resources.resource.Resource'> and service Poisson.


In [112]:
%matplotlib inline
grp = resourceStatsRun.groupby(["serviceScheme", "Resource", "Helpers"])
display(HTML(grp.Rho.agg([np.mean, np.var, np.std]).to_html()))

# grpDescr = grp.describe()
# grpDescr
# # display(HTML(grpDescr.ServeMean.to_html()))
# # display(HTML(grpDescr.Average.to_html()))

# fig, ax = plt.subplots(figsize=(8,6))
# grp.Rho.agg([np.mean]).plot(kind='barh', ax=ax)

In [113]:
%matplotlib inline
grp = resourceStatsRun.groupby(["serviceScheme", "Resource", "Helpers"])
display(HTML(grp.Average.agg([np.mean, np.var, np.std]).to_html()))

# grpDescr = grp.describe()
# grpDescr
# # display(HTML(grpDescr.ServeMean.to_html()))
# # display(HTML(grpDescr.Average.to_html()))

# fig, ax = plt.subplots(figsize=(8,6))
# grp.Average.agg([np.mean]).plot(kind='barh', ax=ax)

In [5]:
lijstpoisson = []
lijstexpo = []

for i in range(1000):
    po = random.expovariate(2)
    pa = np.random.poisson(8/5)
    print(po)
    lijstpoisson.append(po)
    lijstexpo.append(pa)
print(np.mean(lijstpoisson))
print(np.mean(lijstexpo))

0.27564303478464175
0.08569000467503954
0.781497943472319
0.4159544042181474
0.6667151504336933
1.0155631547944555
0.5451054498044076
0.6084484075900936
1.1264807650764432
0.12605087369767418
0.09214449082393791
0.5019681155630527
0.2217045075961715
0.27567534853014275
0.6651389413446762
0.46270818867050456
0.45852800828499385
0.3555993297998512
0.6651455830294382
0.07199210435461877
1.6705153206705277
0.5177824457371579
0.9596897787562763
0.538738722485961
0.7273572523217614
0.8930810550500616
0.986950948705503
1.6745137052531245
0.1812064409235756
0.16063131094389801
0.8922278846041303
0.3065258564364697
0.14836366261793682
0.8833728719124905
0.2369256742326863
0.26295587384953734
0.718099191404133
0.5138415206841446
0.07796438456777739
0.4120699391368748
0.15649522472127397
0.38490903035529234
0.1108419984790977
0.31775366467055915
0.35305061661312975
0.542245527757193
0.036496965316296336
1.6188637203908287
0.03735272090627241
0.6170646521979681
0.24336027439278726
0.33849096319753